In [1]:
import numpy as np
import pymc3 as pm

In [7]:
# Common effect model with 2 causes (c1 and c2) causing an effect e.
# Causes combine using a 'noisy or' rule: two causes independently bring about the effect with probability m.
# r is the likelihood of the causes.
# bg is the probability of the effect coming about without a cause.

ncauses = 2
nvars = ncauses + 1

joints = np.array([[c1, c2, e] for c1 in [0, 1] for c2 in [0, 1] for e in [0, 1]])
resps = (np.array([10.0, 1.0, 3.0, 10.0, 4.0, 10.0, 4.0, 11.0])-1)/10 # converting from their 11-point scale.


def noisy_or(r, m, bg):
    pc = pm.math.switch(joints[:, :-1], r, 1-r)
    pe_absent = (1-bg) * np.prod(1-r*m, axis=1)
    pe = pm.math.switch(joints[:, -1], 1-pe_absent, pe_absent)
    return np.prod(pc, axis=1) * pe

def beta_recoded(name, ratio, confidence, **kwargs):
    beta = confidence / (ratio + 1)
    alpha = confidence - beta
    return pm.Beta(name, alpha=alpha, beta=beta, **kwargs)

with pm.Model() as model:
    r = pm.Beta('r', alpha=1, beta=1, shape=(1, ncauses))
    m = pm.Beta('m', alpha=1, beta=1, shape=(1, ncauses))
    bg = pm.Beta('bg', alpha=1, beta=1)
    pjoints = noisy_or(r, m, bg)
    ratings = beta_recoded('ratings', pjoints, 5, observed=resps)

TypeError: prod() got an unexpected keyword argument 'out'

In [ ]:
with model:
    # draw 500 posterior samples
    trace = pm.sample(5000)

In [ ]:
with model:
    # Run ADVI which returns posterior means, standard deviations, and the evidence lower bound (ELBO)
    v_params = pm.variational.advi(n=50000)